## Montel Carlo examples with numpy

The Monte Carlo method is a technique for numerical approximating quantities that are hard to compute or having no closed-form answer. The basic idea comes from the law of large numbers: if we repeatedly draw random samples according to an underlying distribution, the average output will be a good unbiased estimator of the expected value.

Here we use numpy to demonstrate the Monte Carlo method by three examples.

### First example
We will use Monte Carlo to estimate the value of $\pi$. The idea is that $\sqrt{1-x^2}$ is a quarter of the circle centered at $0$ with radius $1$, so the integral $\int_0^1 4\sqrt{1-x^2} dx$ would be equal to $\pi$. 

In probability term this means that if we denote by $U \sim \textrm{Unif}[0,1]$ the uniform distribution on the interval $[0,1]$, then 

$$ \mathbb{E}\left[4\sqrt{1-U^2} \right] = \pi.$$

If we use a random generator with distribution $\mathrm{Unif}[0,1]$ to generate $n$ i.i.d. points $ U_1, U_2, \cdots, U_n,$ and let $X_i = 4\sqrt{1-U_i^2}$, then the average

$$ \bar{X_n} = \frac{1}{n} \sum_{i=1}^n X_i $$ 

will converge to this integral which happens to be $\pi$.

In [6]:
import numpy as np
import math


First we fix a random seed so we can consistently reproduce the same result. 

The function __gen_unif_pts__ takes an integer input $k > 0$ and outputs a numpy array of $10^k$ points sampled uniformly from $[0,1]$. So the number of points grows exponentially in $k$.

The function __func1__ apply the above math function to the above sampled points.

In [2]:
np.random.seed(0)

def gen_unif_pts(k):
    return np.random.uniform(0,1,10**k)

def func1(ui):
    return 4*(1-ui**2)**(1/2)

Repeat the above procedure for $k=1,...,8$ times. Each time the $\pi$ estimate using $10^k$ points is stored in the list __pi_estimates__.

In [3]:
pi_estimates = []

for k in range(1,9):
    u = gen_unif_pts(k)
    x = func1(u)
    pi_estimates.append(np.sum(x) / 10**k)

Here are the results.

In [4]:
pi_estimates

[2.9531617097666247,
 3.2151047907367825,
 3.1352154197921207,
 3.1548439503506986,
 3.1392348757160287,
 3.1402910297823166,
 3.1418706423044367,
 3.141665847411977]

With the __math__ function we can import the value of $\pi$. We can look at the convergence by calculating the absolute distance between each estimate and the actual value of $\pi$. 

In [5]:
pi = math.pi
pi_error = []
for val in pi_estimates:
    pi_error.append(abs(val-pi))
    
pi_error

[0.18843094382316838,
 0.07351213714698934,
 0.00637723379767241,
 0.013251296760905529,
 0.002357777873764455,
 0.0013016238074765596,
 0.00027798871464357333,
 7.319382218407e-05]

### Second example

This is a variant of the first example. Here, we will approximate $\pi$ using only a fair coin. This amounts to a random output of $0, 1$ with equal probability. We first use numpy's __np.random.randint__ to generate a sequence of $0,1$'s, say, $30$ of them.

In [58]:
digit_array = np.random.randint(0,2,size=30)
digit_array

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0])

We can think of this array of 30 $0,1$'s as a binary number $0.b_1b_2...b_{30}$, which can then be converted into a decimal number by a simple function __binary_to_decimal__.

In [63]:
def decorator_list(fnc):
    def inner(list_of_binary):
        return [fnc(val) for val in list_of_binary]
    return inner


@decorator_list
def binary_to_decimal(num_array):
    array_len = len(num_array)
    num = 0
    for i in range(array_len):
        num += num_array[i] * ((1/2) ** (i+1))
        
    return num
      

Now let's make an array of such numbers. The function __coin_flip_to_binary__ takes an input $n$ and outputs a list of $n$ binary numbers.

In [66]:
def coin_flip_to_binary(n):
    digit_array = np.random.randint(0,2,size=30*n)
    binary_list = []
    for i in range(n):
        binary_list.append(digit_array[30*i:(30*(i+1))])
                           
    return binary_list

In [70]:
np.random.seed(0)

binary_list = coin_flip_to_binary(10**5)
decimal_list = binary_to_decimal(binary_list)

decimal_list[0:5]

[0.4370740931481123,
 0.6780981803312898,
 0.6570075731724501,
 0.7876648996025324,
 0.9837806662544608]

So now we are able to use a fair coin to generate random numbers in the interval $[0,1]$. To approximate $\pi$, we will use a well-known idea: instead of a list of random numbers in $[0,1]$, we can generate a list of 2-tuples in the unit square $[0,1]$. Then, we count the number of points whose distance from the origin is less than 1. As the area of the unit circle in the first quadrant is $\pi/4$, the ratio between the number of such points and the total number of points generated is approximately $\pi/4$.

In [80]:
np.random.seed(1)

# set the number of points in the unit square
n = 10 ** 5

# produce a list of 2n points
# the first half will be the x-coordinates
# the second half will be the y-coordinates
binary_list = coin_flip_to_binary(2*n)

binary_list1 = binary_list[:int(len(binary_list)/2)]
binary_list2 = binary_list[int(len(binary_list)/2):]

decimal_list1 = binary_to_decimal(binary_list1)
decimal_list2 = binary_to_decimal(binary_list2)

unit_sq_list = list(zip(decimal_list1, decimal_list2))

In [82]:
unit_sq_list[:5]

[(0.8108866531401873, 0.8648125613108277),
 (0.24315481819212437, 0.7124165585264564),
 (0.4343401016667485, 0.3216803278774023),
 (0.23230358958244324, 0.1268949992954731),
 (0.4937208453193307, 0.7595623796805739)]

In [86]:
# find the number of points with distance from the origin < 1

pt_in_cir = []
for pt in unit_sq_list:
    if (pt[0] ** 2) + (pt[1] ** 2) < 1:
        pt_in_cir.append(pt)

# approximating pi
(4 * len(pt_in_cir)) / n

3.14788

### Third example

Recall that the number of sampled points grows exponentially in $k$, but even then, the error is converging to $0$ by about a factor of $10$ each iterate, which is rather slow. There are more efficient approximations than Monte Carlo in low dimension integration, but they usually do not generalize to higher dimension, while Monte Carlo does.

Meanwhile, we can use distributions other than $\mathrm{Unif}$ to generate the input values. For example, if we were to approximate the integral
$$ \int_0^\infty e^{\sin(\ln x)) - x^2} dx. $$

Recall that the expoential distribution $\mathrm{Exp}(\lambda)$ has probability distribution function
$$f(x) = \lambda e^{-\lambda x}\quad (x \geq 0),$$
so by rewriting the integral as
$$ \int_0^\infty e^{\sin(\ln x)) - x^2 + x} \quad e^{-x} dx, $$
it can be intepreted as the expectation
$$ \mathbb{E}\left[ e^{\sin(\ln X)) - X^2 + X}\right] $$
with the $\mathrm{Exp}(1)$ distribution.

We can use numpy to generate points with exponential distribution. This time, we use $k=1,...,6$. Then we repeat the above procedure with the new function.

In [6]:
np.random.seed(0)

def gen_exp_pts(k):
    return np.random.exponential(1,10**k)

In [7]:
def func2(ui):
    return  math.exp(math.sin(math.log(ui))-ui**2 +ui)

In [8]:
integral_estimates = []

for k in range(1,7):
    
    u = gen_exp_pts(k)
    x = []
    
    for ui in u:
        x.append(func2(ui))

    
    integral_estimates.append(np.sum(np.array(x)) / 10**k)

Here are the estimates using $n = 10^k$ points for $k=1,...,6$. The actual value of the integral is about 0.65405, but again even with exponentially growing sample points, the convergence by Monte Carlo is still a bit slow.

In [9]:
integral_estimates

[0.7051509089068945,
 0.6828131850369473,
 0.6407661761210932,
 0.6574424007887892,
 0.6528068662672154,
 0.6535148002017594]